In [116]:
import pandas as pd
import numpy as np
import zipfile
import sqlite3

In [2]:
data_sources = {
    # movielens
    'genome_scores'    : './data/ml-25m/genome-scores.csv',
    'genome_tags'      : './data/ml-25m/genome-tags.csv',
    'links'            : './data/ml-25m/links.csv',
    'movies'           : './data/ml-25m/movies.csv',
    'ratings'          : './data/ml-25m/ratings.csv',
    'tags'             : './data/ml-25m/tags.csv',
#     imdb
    'imdb_name_basics' : './data/name.basics.tsv.gz',
    'imdb_title_basics': './data/title.basics.tsv.gz',
    'imdb_ratings'     : './data/title.ratings.tsv.gz',
    'imdb_crew'        : './data/title.crew.tsv.gz',
    'imdb_principals'  : './data/title.principals.tsv.gz',
}

In [ ]:
dfs = {}
for name, path in data_sources.items():
    print(dfs.keys(), end='\r')
    separator = '\t' if 'tsv' in path else ','
    dfs[name] = pd.read_csv(path, sep=separator).replace(to_replace=r'\N', value=np.nan)

In [84]:
for name, path in data_sources.items():
    dfs[name] = dfs[name].dropna()

# connect to database

In [117]:
db_name = './movie_sqlite.db'
conn = sqlite3.connect(db_name)
c = conn.cursor()

In [57]:
# get needed movie ids
sql = """
select id from movies ;
"""
c.execute(sql)
movie_ids = c.fetchall()
needed_movies = [m[0] for m in movie_ids]
needed_movies[:5]  # check first 5

['tt0000012', 'tt0000417', 'tt0000439', 'tt0004972', 'tt0006333']

In [58]:
df_needed_movies = pd.DataFrame(needed_movies)
df_needed_movies.columns = ['movie_id']
df_needed_movies.head()

,movie_id
0,tt0000012
1,tt0000417
2,tt0000439
3,tt0004972
4,tt0006333


In [175]:
df_needed_movies.shape

(13795, 1)

In [59]:
# actors table
# directors table

In [60]:
needed_movies_joined_to_directors = pd.merge(df_needed_movies,
                                             dfs['imdb_crew'],
                                             left_on='movie_id',
                                             right_on='tconst',
                                             how='left')
needed_movies_joined_to_directors.head()

,movie_id,tconst,directors,writers
0,tt0000012,tt0000012,"nm0525910,nm0525908",NaN
1,tt0000417,tt0000417,nm0617588,"nm0617588,nm0894523,nm0920229"
2,tt0000439,tt0000439,nm0692105,"nm1145809,nm0692105"
3,tt0004972,tt0004972,nm0000428,"nm0228746,nm0000428,nm0940488"
4,tt0006333,tt0006333,nm0665737,"nm0894523,nm0665737"


In [205]:
unique_movies = needed_movies_joined_to_directors.movie_id.unique()
unique_movies

array(['tt0000012', 'tt0000417', 'tt0000439', ..., 'tt9412098',
       'tt9495224', 'tt9573980'], dtype=object)

In [201]:
needed_movies_joined_to_directors.movie_id.nunique()

13790

In [176]:
needed_movies_joined_to_directors.directors.nunique()

5744

In [30]:
np.nan

nan

In [61]:
# needed_movies_joined_to_directors.loc[:,'writers'] = np.where(needed_movies_joined_to_directors.writers == r'\N',
#                                                               np.nan,
#                                                               needed_movies_joined_to_directors.writers)

In [62]:
needed_movies_joined_to_directors = (needed_movies_joined_to_directors[~
                               needed_movies_joined_to_directors.directors.isna()])

In [63]:
needed_movies_joined_to_directors[needed_movies_joined_to_directors.directors.isna()]

,movie_id,tconst,directors,writers


In [64]:
needed_movies_joined_to_directors.shape

(13790, 4)

In [65]:
all_needed_directors = pd.Series(needed_movies_joined_to_directors.directors.unique())

In [178]:
all_needed_directors.shape

(5744,)

In [66]:
all_needed_directors.head()

0    nm0525910,nm0525908
1              nm0617588
2              nm0692105
3              nm0000428
4              nm0665737
dtype: object

In [67]:
cleaned_all_needed_directors = []
for pair in [person.split(',') for person in all_needed_directors]:
    for item in pair:
        cleaned_all_needed_directors.append(item)

In [68]:
cleaned_all_needed_directors = pd.DataFrame(np.array(cleaned_all_needed_directors), columns=['nconst'])
cleaned_all_needed_directors.head()

,nconst
0,nm0525910
1,nm0525908
2,nm0617588
3,nm0692105
4,nm0000428


In [69]:
cleaned_all_needed_directors.shape

(7295, 1)

In [81]:
'nm5278146' in cleaned_all_needed_directors.nconst

False

In [202]:
dfs['imdb_crew'].head()

,tconst,directors,writers
8,tt0000009,nm0085156,nm0085156
34,tt0000036,nm0005690,nm0410331
74,tt0000076,nm0005690,nm0410331
89,tt0000091,nm0617588,nm0617588
106,tt0000108,nm0005690,nm0410331


In [94]:
dfs['imdb_crew'][dfs['imdb_crew'].writers.isna()]

,tconst,directors,writers


In [96]:
cleaned_all_needed_directors[cleaned_all_needed_directors.nconst.isna()]

,nconst


In [102]:
# * NOTE some directors exist in the directors field for movies but in the imdb crew are not found
#  using this we can find these cases how='left')
# 618 directors dont exist in the imdb crew dataset

In [179]:
cleaned_all_needed_directors.head()

,nconst
0,nm0525910
1,nm0525908
2,nm0617588
3,nm0692105
4,nm0000428


In [180]:
dfs['imdb_crew'].head()

,tconst,directors,writers
8,tt0000009,nm0085156,nm0085156
34,tt0000036,nm0005690,nm0410331
74,tt0000076,nm0005690,nm0410331
89,tt0000091,nm0617588,nm0617588
106,tt0000108,nm0005690,nm0410331


In [208]:
directors_and_movies = pd.merge(cleaned_all_needed_directors, 
                                dfs['imdb_crew'],
                                left_on='nconst',
                                right_on='directors',
                                how='inner')
directors_and_movies.head()

,nconst,tconst,directors,writers
0,nm0525910,tt1167665,nm0525910,nm0525910
1,nm0525910,tt1736627,nm0525910,nm0525910
2,nm0525910,tt2461054,nm0525910,nm0525910
3,nm0525910,tt6231346,nm0525910,nm0525910
4,nm0617588,tt0000091,nm0617588,nm0617588


In [214]:
directors_and_movies.tconst.nunique()

156642

In [216]:
# filter this set down by only the movies we have access to.
directors_and_movies_filtered = directors_and_movies[directors_and_movies.tconst.isin(unique_movies)]
directors_and_movies_filtered.tconst.nunique()

12513

In [217]:
# check how many movies each director was in
s = directors_and_movies.groupby('nconst').count()['tconst'].sort_values(ascending=False)
s.head()

nconst
nm0455741    1788
nm0814716    1570
nm0005062    1540
nm1853544    1080
nm0400958     904
Name: tconst, dtype: int64

In [218]:
s.tail()

nconst
nm1390562    1
nm0854418    1
nm0859016    1
nm0859877    1
nm0325533    1
Name: tconst, dtype: int64

In [219]:
directors_movie_counts = s.reset_index()
directors_movie_counts[directors_movie_counts['nconst'] == 'nm0455741']

,nconst,tconst
0,nm0455741,1788


In [220]:
# EDA this is the average number of movies each director in our dataset directed
s.mean()

34.513653399964305

In [221]:
directors_and_movies = directors_and_movies[['directors', 'tconst']]
directors_and_movies.columns = ['directors', 'movies']

In [222]:
# load movies, ids, releveance into df
# join this with directors and movies on movies id
# aggregate on [directors, movies] and apply aggregate function (mean) (or weighted based on rating)

In [223]:
sql = '''
select *
from tag_relevance
join movies
on fk_id = id
limit 50000000
'''
c
df_movie_tags = pd.read_sql(sql, con=conn)
df_movie_tags.head()

,fk_id,tag_id,relevance,id,kind,primary_title,original_title,release_year,runtime_minutes,genres
0,tt0000012,1,0.04500,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
1,tt0000012,2,0.04225,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
2,tt0000012,3,0.03475,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
3,tt0000012,4,0.03750,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
4,tt0000012,5,0.21475,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N


In [224]:
# df_movie_tags.groupby('fk_id').count().tag_id.unique()

In [225]:
# NOTE: fix nans from being joined above
# for now just remove

In [226]:
# directors_and_movies = directors_and_movies[~directors_and_movies.tconst.isna()]

In [227]:
directors_and_movies.head()

,directors,movies
0,nm0525910,tt1167665
1,nm0525910,tt1736627
2,nm0525910,tt2461054
3,nm0525910,tt6231346
4,nm0617588,tt0000091


In [228]:
directors_and_movies[directors_and_movies['directors'] == 'nm0000008']

,directors,movies
39150,nm0000008,tt0055257


In [229]:
# directors_and_movies.loc[:,'movie_id'] = (directors_and_movies.loc[:,'tconst'].str[2:]).astype(int)
# directors_and_movies.head()

In [230]:
# df_movie_tags.head()

In [231]:
df_movie_tags.head()

,fk_id,tag_id,relevance,id,kind,primary_title,original_title,release_year,runtime_minutes,genres
0,tt0000012,1,0.04500,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
1,tt0000012,2,0.04225,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
2,tt0000012,3,0.03475,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
3,tt0000012,4,0.03750,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N
4,tt0000012,5,0.21475,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896,1,\N


In [232]:
directors_and_movies.head()

,directors,movies
0,nm0525910,tt1167665
1,nm0525910,tt1736627
2,nm0525910,tt2461054
3,nm0525910,tt6231346
4,nm0617588,tt0000091


In [233]:
directors_and_movies.directors.nunique()

5603

In [249]:
directors_and_movies.groupby('directors').count().sort_values('movies', ascending=False)

,movies
directors,
nm0455741,1788
nm0814716,1570
nm0005062,1540
nm1853544,1080
nm0400958,904
...,...
nm0803020,1
nm0534681,1
nm0681553,1


In [247]:
# directors_and_movies.groupby('movies').count().sort_values('directors', ascending=False)

,directors
movies,
tt3711804,13
tt0301201,13
tt0151830,13
tt0151857,13
tt0055089,13
...,...
tt0587267,1
tt0587266,1
tt0587265,1


In [235]:
df_movie_tags[df_movie_tags['fk_id'] == 'tt0032544']

,fk_id,tag_id,relevance,id,kind,primary_title,original_title,release_year,runtime_minutes,genres


In [259]:
merged_df = pd.merge(directors_and_movies, df_movie_tags, left_on='movies', right_on='fk_id', how='inner')
# merged_df[~merged_df.tag_id.isna()].head()
merged_df.shape

(494064, 12)

In [260]:
merged_df[merged_df['directors'] == 'nm0455741']

,directors,movies,fk_id,tag_id,relevance,id,kind,primary_title,original_title,release_year,runtime_minutes,genres


In [252]:
merged_df.shape

(687006, 12)

In [253]:
directors_and_movies.directors.nunique() * 1128

6320184

In [263]:
directors_tag_relevance = merged_df.groupby(['directors', 'tag_id'])['relevance'].mean().reset_index()
directors_tag_relevance

,directors,tag_id,relevance
0,nm0000008,1,0.04475
1,nm0000008,10,0.01650
2,nm0000008,100,0.61000
3,nm0000008,1000,0.04275
4,nm0000008,1001,0.04375
...,...,...,...
277483,nm0958387,995,0.11225
277484,nm0958387,996,0.06875
277485,nm0958387,997,0.05275
277486,nm0958387,998,0.14750


In [ ]:
15584448
current rows
updated rows
15861936


In [ ]:
fk_id text NOT NULL,
    tag_id name text NOT NULL,
    relevance real NOT NULL

In [276]:
directors_tag_relevance.columns = ['fk_id', 'tag_id', 'relevance']

In [277]:
directors_tag_relevance.to_sql('tag_relevance', conn, if_exists='append', index=False)

In [278]:
15861936 - 15584448

277488

In [279]:
directors_tag_relevance.shape

(277488, 3)

In [266]:
directors_tag_relevance[directors_tag_relevance['directors'] == 'nm0958387'].relevance.max()

0.993

In [267]:
directors_tag_relevance[directors_tag_relevance['directors'] == 'nm0958387'][directors_tag_relevance[directors_tag_relevance['directors'] == 'nm0958387']['relevance']== 0.993]

,directors,tag_id,relevance
276856,nm0958387,43,0.993


In [272]:
directors_and_movies[directors_and_movies.directors == 'nm0958387']

,directors,movies
72126,nm0958387,tt0099653
72127,nm0958387,tt0113071
72128,nm0958387,tt0250687
72129,nm0958387,tt0099653
72130,nm0958387,tt0113071
72131,nm0958387,tt0250687
72132,nm0958387,tt0099653
72133,nm0958387,tt0113071
72134,nm0958387,tt0250687


In [273]:
dfs.keys()

dict_keys(['genome_scores', 'genome_tags', 'links', 'movies', 'ratings', 'tags', 'imdb_name_basics', 'imdb_title_basics', 'imdb_ratings', 'imdb_crew', 'imdb_principals'])

In [275]:
dfs['imdb_principals']

,tconst,ordering,nconst,category,job,characters


In [274]:
dfs['imdb_crew']

,tconst,directors,writers
8,tt0000009,nm0085156,nm0085156
34,tt0000036,nm0005690,nm0410331
74,tt0000076,nm0005690,nm0410331
89,tt0000091,nm0617588,nm0617588
106,tt0000108,nm0005690,nm0410331
...,...,...,...
6615770,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
6615771,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
6615772,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
6615773,tt9916856,nm10538645,nm6951431


In [268]:
dfs['genome_tags'][dfs['genome_tags'].tagId == 43]

,tagId,tag
42,43,alien


In [258]:
directors_tag_relevance[directors_tag_relevance==0.99925]

,directors,tag_id,relevance
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
277483,NaN,NaN,NaN
277484,NaN,NaN,NaN
277485,NaN,NaN,NaN
277486,NaN,NaN,NaN


In [245]:
directors_tag_relevance = pd.DataFrame(directors_tag_relevance).reset_index()
directors_tag_relevance.directors.nunique()

246

In [246]:
pd.DataFrame(directors_tag_relevance)

,index,directors,tag_id,relevance
0,0,nm0000008,1,0.04475
1,1,nm0000008,10,0.01650
2,2,nm0000008,100,0.61000
3,3,nm0000008,1000,0.04275
4,4,nm0000008,1001,0.04375
...,...,...,...,...
277483,277483,nm0958387,995,0.11225
277484,277484,nm0958387,996,0.06875
277485,277485,nm0958387,997,0.05275
277486,277486,nm0958387,998,0.14750


In [76]:
dfs.keys()

dict_keys(['genome_scores', 'genome_tags', 'links', 'movies', 'ratings', 'tags', 'imdb_name_basics', 'imdb_title_basics', 'imdb_ratings', 'imdb_crew', 'imdb_principals'])

In [78]:
dfs['imdb_name_basics'].head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,\N,\N,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0050419,tt0043044"
1,nm0000002,Lauren Bacall,\N,\N,"actress,soundtrack","tt0071877,tt0117057,tt0037382,tt0038355"
2,nm0000003,Brigitte Bardot,\N,\N,"actress,soundtrack,producer","tt0059956,tt0057345,tt0054452,tt0049189"
3,nm0000004,John Belushi,\N,\N,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,\N,\N,"writer,director,actor","tt0050976,tt0069467,tt0083922,tt0050986"


In [81]:
needed_directors_and_names = pd.merge(cleaned_all_needed_directors,dfs['imdb_name_basics'],
                                      left_on='nconst', right_on='nconst', how='left' )
needed_directors_and_names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0005139,Mike Leigh,\N,\N,"director,writer,actor","tt1431181,tt0117589,tt0107653,tt1045670"
1,nm0504802,Sheldon Lettich,\N,\N,"writer,director,producer","tt0100029,tt0095956,tt3369806,tt0101764"
2,nm0669004,Richard Pearce,\N,\N,"director,cinematographer,producer","tt0406429,tt0079261,tt1286537,tt0082508"
3,nm0720000,Norman René,\N,\N,"director,producer","tt0105165,tt0176357,tt0114241,tt0100049"
4,nm0002132,Amy Heckerling,\N,\N,"writer,director,producer","tt0115137,tt0097778,tt0083929,tt3973820"


In [84]:
needed_directors_and_names = needed_directors_and_names[['nconst','primaryName']]

In [87]:
dfs['directors'] = needed_directors_and_names
dfs['directors_relevence'] = directors_tag_relevance

In [91]:
dfs['directors']

,nconst,primaryName
0,nm0005139,Mike Leigh
1,nm0504802,Sheldon Lettich
2,nm0669004,Richard Pearce
3,nm0720000,Norman René
4,nm0002132,Amy Heckerling
...,...,...
7290,nm0540330,Bruce Malmuth
7291,nm0822582,Richard Stanley
7292,nm0663489,James D. Parriott
7293,nm0955443,Yimou Zhang


In [93]:
# 
#
directors_table = """
CREATE TABLE IF NOT EXISTS directors (
    id text PRIMARY KEY,
    name text
);
"""

director_relevence_table = """
CREATE TABLE IF NOT EXISTS director_tag_relevance (
    director_id integer,
    tag_id name text NOT NULL,
    relevance real NOT NULL
);
"""
table_sql = {
    'directors'                 : directors_table,
    'directors_relevence'       : director_relevence_table,
}

table_data_source = {
    'directors'                 : 'directors',
    'directors_relevence'       : 'directors_relevence',
}

for table_name in table_sql:
    # create table
    c.execute(table_sql[table_name])
    # populate table with data
    df = dfs[table_data_source[table_name]]
    df.to_sql(table_sql[table_name], conn, if_exists='append', index=False)

In [ ]:
# get needed movie ids
sql = """
select id from movies ;
"""
c.execute(sql)
movie_ids = c.fetchall()
needed_movies = [m[0] for m in movie_ids]
needed_movies[:5]  # check first 5

In [20]:
# director aggregated tag score
# average for each tag for all of the movies they directed


In [21]:
# get needed movie ids
sql = """
select * from tags ;
"""
c.execute(sql)
tag_ids = c.fetchall()
all_tags = [t for t in tag_ids]
all_tags[:5]  # check first 5

[(1, '007'),
 (2, '007 (series)'),
 (3, '18th century'),
 (4, '1920s'),
 (5, '1930s')]

In [22]:
# get needed movie ids
sql = """
select * from tag_relevance;
"""
c.execute(sql)
movie_ids = c.fetchall()
needed_movies = [m for m in movie_ids]
needed_movies[:5]  # check first 5
# movie, tag, relevance

[(1, '1', 0.02875),
 (1, '2', 0.023749999999999997),
 (1, '3', 0.0625),
 (1, '4', 0.07574999999999997),
 (1, '5', 0.14075)]

In [58]:
dfs.keys()

dict_keys(['genome_scores', 'genome_tags', 'links', 'movies', 'ratings', 'tags', 'imdb_name_basics', 'imdb_title_basics', 'imdb_ratings', 'imdb_crew', 'imdb_principals'])

In [ ]:
dfs['']

In [ ]:
# director_id, tag_id, relevence_score
# 1, 1, .5
# 1, 2, .2
# ....

In [ ]:
# actor_id, tag_id, relevence_score
# 1, 1, .5
# 1, 2, .2
# ....

In [ ]:
s.head()

In [15]:
actors

actor_id, name, tag_id, tag_value

NameError: name 'actors' is not defined